# Featurization

# Assignments:

1. Add another feature called  Preferential Attachment  with followers and followees data of vertex. you can check about Preferential Attachment in below link
http://be.amazd.com/link-prediction/ <br>
2. Add  feature called svd_dot. you can calculate svd_dot as Dot product between sourse node svd and destination node svd features.  you can read about this in below pdf 
https://storage.googleapis.com/kaggle-forum-message-attachments/2594/supervised_link_prediction.pdf<br>
3. Tune hyperparameters for XG boost with all these features and check the error metric.

### 1.  Preferential Attachment

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from tqdm import tqdm

In [5]:
def feature_extration(data):
    
    # Preprosseing
    
    node_follower =dict()
    for idx,rows in tqdm(data.iterrows(),total=len(data)):
        node_follower[rows['source_node']] = rows['num_followers_s']
    features = []
    
    for idx,rows in tqdm(data.iterrows(),total=len(data)):
        feat1 = [rows['num_followees_s']*rows['num_followees_d'] ]
        if node_follower.get(rows['destination_node']) is None:
            feat1.append(0)
        else:
            feat1.append(node_follower[rows['destination_node']]*rows['num_followers_s'])
        u_s = np.array(rows[['svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4','svd_u_s_5', 'svd_u_s_6']])
        u_d = np.array(rows[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3','svd_u_d_4', 'svd_u_d_5', 'svd_u_d_6']])
        v_s = np.array(rows[['svd_v_s_1', 'svd_v_s_2','svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6']])
        v_d = np.array(rows[['svd_v_d_1','svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5', 'svd_v_d_6']])
        feat2 = [np.dot(u_d,u_s)/(np.linalg.norm(u_d)*np.linalg.norm(u_s)),np.dot(v_s,v_d)/(np.linalg.norm(v_d)*np.linalg.norm(v_s))]# should we normalize
        feat1.extend(feat2)
        features.append(feat1)
    pd_features = pd.DataFrame(features,columns=["preferentail_followees","preferential_follower","svd_dot_u","svd_dot_v"])
    
    return pd.concat([data,pd_features],axis=1)  


In [3]:
df_final_train = pd.read_hdf('data/fea_sample/storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = pd.read_hdf('data/fea_sample/storage_sample_stage4.h5', 'test_df',mode='r')

In [6]:
feat = feature_extration(df_final_train)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100002/100002 [02:40<00:00, 623.24it/s]


In [9]:
feat['indicator_link'].value_counts()

1    50050
0    49952
Name: indicator_link, dtype: int64

In [10]:
test = feature_extration(df_final_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50002/50002 [01:21<00:00, 617.18it/s]


In [11]:
test['indicator_link'].value_counts()

0    25046
1    24956
Name: indicator_link, dtype: int64

In [12]:
hdf = pd.HDFStore('./data/fea_sample/storage_sample_stage5.h5')
hdf.put('train_df',feat, format='table', data_columns=True)
hdf.put('test_df',test, format='table', data_columns=True)
hdf.close()